# **최적의 프롬프트 확보를 위한 테스트 파일**

1. 목적:  Prompt 형식과 예시(few-shot 포함 여부)에 따른 성능 변화 측정
2. 실험 내 예시

    > Zero-shot vs Few-shot Prompt

    > Chain-of-Thought Prompt 유무

    > Instruction 단순화 or 세분화

3. 측정 포인트
- 답변 길이 조절
- Faithfulness (정확히 근거 문서를 반영하는가)
- Answer Relevance


## **환경설정(실험동안 변경 X)**
- DB 연결
- 모델 설정
- 프롬프트 설정
- 임베딩 모델 설정
- 벡터스토어 설정

In [99]:
# 테스트를 위한 셋업
import os
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain.schema import Document
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 환경 변수 로드 (.env)
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Pinecone 인덱스 설정
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "hufs-chatbot"

# 모델 설정
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo"  
)

# 임베딩 모델, 벡터스토어 설정
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")  

vectorstore = PineconeVectorStore.from_existing_index(
    index_name="hufs-chatbot",
    embedding=embedding,
     text_key="page_content" 
)

# 검색기 에 따라 변경될 내용
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

## **프롬프트 설정**
- 프롬프트 엔지니어링 기법을 적용해가며 최적의 성능을 내는 프롬프트 설정
- 각 내용, 조합별 성능이나 문제점 등 기록 필수

In [100]:
# 프롬프트 정의
prompt = PromptTemplate.from_template("""
너는 한국외국어대학교의 학생 및 교직원들을 도와주는 AI 챗봇이야.

학생들은 학과명이나 수업 관련 표현을 축약하거나 구어체로 말할 수 있어.  
다음과 같은 규칙을 참고해서 질문을 해석해야 해.  
항상 문서에서 제공된 정보만 바탕으로 정확하게, 공손한 높임말을 사용해서 답변해.  
정확한 유추가 가능한 경우에는 예시를 참고해서 논리적으로 추론해도 괜찮아. 하지만 정보가 불충분하면 모른다고 답해야 해.


[축약어 해석 규칙]
- "일통" → "일본어통번역학과"
- "글스산" → "글로벌스포츠산업학부"
- "전필" → "전공필수", "전선" → "전공선택"
- "유고" → "유고결석", "공결" → "공식결석"
- "[언어명]통" → "[언어명]어통번역학과"로 일반화 가능


📥 [사용자 질문]  
{question}

📄 [참고 문서]  
{context}

💭 [답변을 위한 생각]  
(질문의 의미를 해석하고 관련 정보를 연결합니다.)

💬 [답변]  
(공손하고 정확하게 답변합니다.)
""")

few-shot + instruction -> 축약어와 답변추론 부분에서 괜찮지만, 교수님들에 따라 답변이 다른 경우나 추가적인 도움이 필요한 상황에서의 대답이나 답변길이 부분에서 부족함

In [ ]:
prompt = PromptTemplate.from_template("""
너는 한국외국어대학교 학생들의 학사 관련 질문에 답변하는 AI 챗봇이야.

- 사용자는 일상적인 표현, 축약어, 은어 등을 쓸 수 있어.
- 항상 문서에서 제공한 정보에 기반하여 '정확하고 공손하게' 답변해.
- 질문자의 핵심 의도에 집중해, 관련 없는 부가정보는 생략해.
- 질문을 받으면 먼저 '의미를 해석하고', 관련 문서에서 '어떤 근거로 어떤 판단을 할 수 있는지' 논리적으로 연결해서 답변해.
- 문서에 정보가 부족하면 "담당 교수님께 문의하라"는 안내로 마무리해.
- 항상 끝에 "혹시 더 도와드릴까요?" 와 비슷한 '부드러운 후속 안내'를 덧붙여.
- 질문에 포함된 표현이 한국어일 경우, 동일한 의미의 영어 표현도 함께 고려해해  
예: “딥러닝” → “Deep Learning”, “데베” → “데이터베이스” →  “Data Base ”
가능하면 질문에서 추출된 의미를 한국어/영어 키워드 모두로 확장해서 관련 문서를 찾아봐봐.

---

[축약어 해석 규칙]
- "일통" → "일본어통번역학과"
- "글스산" → "글로벌스포츠산업학부"
- "전필" → "전공필수", "전선" → "전공선택"
- "유고" → "유고결석", "공결" → "공식결석"
- "[언어명]통" → "[언어명]어통번역학과"로 일반화 가능
                                    
---

📘 [예시 응답 패턴]

Q: 감기로 병원 다녀왔는데 진료확인서로 유고결석 가능해?  
💭 (생각) ‘감기’는 병결 사유이고, '진료확인서'는 증빙 서류임. 유고결석은 병원급 이상 서류를 요구함.  
💬 감기로 병원에 다녀오신 경우, 일반 의원에서 발급한 진료확인서는 인정되지 않을 수 있습니다.  
병원급 이상의 의료기관에서 받은 진료확인서를 제출하시면 유고결석 처리가 가능할 수 있습니다.  
정확한 기준은 과목 담당 교수님께 확인해 주세요. 혹시 더 도와드릴까요?

Q: 일통 전필 뭐야?  
💭 (생각) '일통'은 '일본어통번역학과', '전필'은 전공필수 과목을 의미함.  
💬 일본어통번역학과의 전공필수 과목은 다음과 같습니다...

Q: 취업계 내면 결석 인정되나요?  
💭 (생각) 취업계는 공결 사유에 해당되기도 하지만, 교수 재량에 따라 달라질 수 있음.  
💬 일부 과목에서는 취업계 제출 시 공결로 인정되기도 합니다.  
다만 교수님에 따라 판단 기준이 다르니 꼭 수업 담당 교수님께 먼저 문의하시기 바랍니다. 혹시 더 도와드릴까요?

---

📥 사용자 질문:  
{question}

📄 참고 문서:  
{context}

💭 (질문 해석 및 내부 판단):  
(질문의 의미를 해석하고 관련 정보를 연결하는 사고 과정을 간단히 서술)

💬 (최종 응답):  
(문서 기반으로 논리적인 흐름을 따라 공손하고 명확하게 응답)
""")

few-shot + instruction + CoT -> instruction에 더 디테일한 지시와 CoT 기법을 통해 추가 질문 및 교수에 따라 답변이 달라지는 상황에 대응하게 함

## 체인
- 변경 X 순서도 retriever 설정 뒤로 오도록 할 것

In [102]:
# 체인 구성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## **실험**
- 질문은 기존에 문제를 갖고 있던 질문과 선정된 질문 들로 구성
- 답변 결과 해석: 정확히 답변 했는지, 정확한 답변이 아니라면 어떤 문제가 있었고 어떻게 개선해야 하는지

In [103]:
# 질문 - 답변 확인
question = "입원으로 인한 병결은 유고결석 처리가 돼?" # 질문 입력
response = chain.invoke(question)
print(response)

입원으로 인한 병결은 유고결석 처리가 가능합니다. 다만, 의원에서 발급받은 진료확인서는 인정되지 않으며, 병원급 의료 기관에서 발급받은 증빙서류를 제출해야 합니다. 입원으로 인한 병결은 법정감염병 확인이 필요하며, 격리(치료) 기간 또는 감염우려 기간 동안의 증빙서류를 담당교수에게 제출하셔야 합니다. 자세한 사항은 담당 교수님께 문의하시기 바랍니다. 혹시 더 도와드릴까요?


 축약어도 잘 추론하여 정확한 정보전달을 하고, 교수님마다 달라질 수 있는 상황도 잘 대응함

In [104]:
# 질문 - 답변 확인
question = "교생실습을 나갈때 출석은 어떻게 처리돼?" # 질문 입력
response = chain.invoke(question)
print(response)

교생실습을 나갈 때 출석은 해당 교직담당자가 발급한 확인서를 담당 교수에게 제출하여 처리됩니다. 교생실습 기간 동안의 출석 처리 방법은 교직담당자의 지침에 따라 결정되니, 꼭 해당 교직담당자와 담당 교수에게 문의하시기 바랍니다. 담당 교수의 지침에 따라 출석이 인정될 수 있습니다. 혹시 더 도와드릴까요?


 예시에 없는 경우에도 정확한 답변을 출력함

In [105]:
# 질문 - 답변 확인
question = "글스산 졸업하면 뭐해?" # 질문 입력
response = chain.invoke(question)
print(response)

글로벌스포츠산업학부를 졸업하면 다양한 진로가 가능합니다. 대사관, 정부 산하 공기업 및 준정부기관, 국내 및 해외 대기업, 방송사, 항공사, 언론사, 광고·PR사, IT 기업 등 다양한 분야에서 활동할 수 있습니다. 또한 대학교수, 연구원, 석사 및 박사 진학, 그리고 창업 등의 진로도 선택할 수 있습니다. 자세한 내용은 졸업 후 진로를 결정할 때 담당 교수님과 상담하시는 것이 좋습니다. 혹시 더 도와드릴까요?


축약어의 해석은 하였지만 글스산이 아닌 수학과의 문서를 가져와 답변을 함 
개선 -> 검색기와 조합으로 해결가능 할 것으로 보임

In [106]:
# 질문 - 답변 확인
question = "월요일 7, 8교시에 들을 수 있는 교양 있으면  알려줘." # 질문 입력
response = chain.invoke(question)
print(response)

월요일 7, 8교시에 들을 수 있는 교양 과목은 다음과 같습니다:
1. [신화와종교] - 교수: 강미라 - 강의실 2303-3호
2. [종교철학입문] - 교수: 조명동 - 강의실 2503호

위 두 과목 중에서 선택하실 수 있습니다. 추가적인 도움이 필요하시면 언제든지 물어봐 주세요. 혹시 더 도와드릴까요?


과잉 정보 -	교수 이름, 강의실, 온라인 여부 등 전부 포함
문장 중복 -	“수강하실 수 있습니다”가 이미 앞 내용에서 중복됨
핵심 강조 부족 -	“교양 과목 여부”나 “시간대 매칭”이 명확하지 않음

*개선방안 -  추가 Instruction: 질문자의 핵심 의도에 집중해, 관련 없는 부가정보는 생략해

In [107]:
# 질문 - 답변 확인
question = "졸업하기 전에 무슨 영어성적 제출해야 된다고 했던거 같은데 그건 뭐야?"# 질문 입력
response = chain.invoke(question)
print(response)

졸업 전에 제출해야 하는 영어성적은 외국어 인증 신청서입니다. 졸업 예정자는 최종 학기 말까지 외국어 인증 신청서를 제출해야 합니다. 제출서류로는 외국어인증 신청서와 유효기간 내의 인증 시험 성적이 필요합니다. 제출은 학생의 전공 소속 단과대학 담당에게 E-mail로 하시면 됩니다. 자세한 내용은 담당 교수님께 문의하시기 바랍니다. 혹시 더 도와드릴까요?


무슨 영어성적을 외국어 인증 신청서로 이해한 후 제출 안내까지의 답변을 생성함함

In [108]:
# 질문 - 답변 확인
question = "나 정보통신공학과인데 혹시 졸업시험 대신 졸업논문을 선택해도 될까? 논문을 쓰게 되면 기계학습 관련해서 작성하고 싶은데 이와 관련해서 상담할 수 있는 교수님이 계시면 이메일 좀 알려줘." # 질문 입력
response = chain.invoke(question)
print(response)

졸업시험 대신 졸업논문을 선택할 수 있는지에 대해 문서에 명확한 언급이 없습니다. 따라서, 이에 대한 정확한 판단은 정보통신공학과의 담당 교수님께 문의하시는 것이 좋습니다. 논문을 작성하고 싶은 경우, 정보통신공학과의 한희일 교수님께 이메일을 통해 상담을 요청하실 수 있습니다. 교수님의 이메일은 hihahn@hufs.ac.kr이며, 연구실은 백년관 721호에 위치해 있습니다. 교수님께서 상담 가능한지 확인하시고 자세한 안내를 받아보시기 바랍니다. 혹시 더 도와드릴까요?


In [109]:
# 질문 - 답변 확인
question = "종합설계 보강일 언제야?" # 질문 입력
response = chain.invoke(question)
print(response)

제1학기 보강주간은 2025년 6월 9일(월)부터 6월 13일(금)까지 진행됩니다. 종합설계 보강일은 해당 기간 중 어떤 날짜에 해당하는지 확인하시면 됩니다. 자세한 사항은 학사일정을 참고하시기 바랍니다. 담당 교수님께 문의하시면 더 정확한 안내를 받으실 수 있습니다. 혹시 더 도와드릴까요?


학사공지만 참고하고 강의시간표를 참고하지는 못함

*개선방안 - 참고문서를 늘려서 해결가능

In [110]:
# 질문 - 답변 확인
question = "AI융합전공 외국어인증 신청서 제출 이메일 주소가 뭐고, 언제까지 제출해야해?" # 질문 입력
response = chain.invoke(question)
print(response)

외국어인증 신청서 제출 이메일 주소는 haksa7@hufs.ac.kr이며, 제출 마감일은 재학기간 동안 1회, 외국어 인증 신청서를 제출하고자 하는 학기 말까지입니다. 졸업예정자는 졸업하고자 하는 최종 학기 말까지 반드시 제출해야 합니다. 자세한 내용은 담당 교수님께 문의하시기 바랍니다. 혹시 더 도와드릴까요?


In [112]:
# 질문 - 답변 확인
question = "딥러닝이 연구분야인 교수님들의 소속 단과대와 학부를 알고 싶어." # 질문 입력
response = chain.invoke(question)
print(response)

딥러닝 연구분야의 교수님들은 언어인지과학과에 소속되어 있습니다.  
딥러닝과 관련된 학부는 따로 명시되어 있지 않습니다.  
더 자세한 정보를 원하시면 해당 교수님들에게 직접 문의하시거나 관련 부서에 문의해 주세요.  
혹시 더 도와드릴까요?


딥러닝을 한글의 '딥러닝'으로만 해석하여 한사람의 정보만 가져옴

*개선방안: insruction에 추가 

In [114]:
# 질문 - 답변 확인
question = "후기이중이 뭐야?" # 질문 입력
response = chain.invoke(question)
print(response)

후기이중이란 후기이중전공을 의미합니다. 후기이중전공은 9학기 추가학기 수강신청이 가능하며, 탈락 시 후기이중전공이 불가능합니다. 현재 7~8학기 이수 중이거나 후기이중전공을 희망하는 전공을 부전공으로 이수 중인 학생이 신청할 수 있습니다. 신청 기간은 4월 14일부터 4월 18일까지이며, 신청은 인터넷 종합정보시스템에서 가능합니다. 추가로 자세한 사항은 해당 학과 학과장실이나 학과 홈페이지를 참조하시기 바랍니다. 담당 교수님께 문의하시면 더 자세한 안내를 받을 수 있습니다. 혹시 더 도와드릴까요?


## **검색된 문서 확인**

In [115]:
# 검색된 문서 확인
docs = vectorstore.similarity_search(question, k=5) # 검색 문서 수는 검색기에서 설정한 문서 수와 통일

for doc in docs:
    print(doc.metadata)          # 문서 메타정보 (예: source, page 등)
    print(doc.page_content)  # 문서 내용 미리보기

{'creationdate': '2024-06-21T10:55:17+09:00', 'creator': 'Adobe InDesign 16.4 (Macintosh)', 'doc_type': '전공가이드북', 'moddate': '2024-06-21T11:00:00+09:00', 'page': 14.0, 'page_label': '15', 'producer': 'Adobe PDF Library 16.0', 'source': 'data/major_guide_2025.pdf', 'total_pages': 37.0, 'trapped': '/False'}
을까요?
A3.  전쟁 이후 우크라이나어의 위상은 높아지고 사용 빈도는 증가할 것
입니다. 그리고 우크라이나의 복구 사업에 우리 기업들이 많이 참
여할 것으로 예상되며, 우리 학생들이 전공을 살려 통역, 무역, 비
즈니스, 공공서비스, 연구 등 여러 분야에서 활발히 일할 수 있을 
것입니다.
{'author': '학사종합지원센터(글로벌)', 'date': '2025.01.07', 'doc_type': '학사공지', 'ntt_id': '214822', 'title': '[글로벌] 2025년 전기(2월 졸업) 졸업예비사정 결과 조회 안내', 'url': 'https://www.hufs.ac.kr/bbs/hufs/2181/214822/artclView.do'}
| (기타) | 후기이중전공 | 후기이중전공(진입) 학생은 9학기 추가학기 수강신청 가능 탈락시 후기이중전공 불가 | 기타 특이사항 명시 ※ 해당사항이 없는 학생은 표시되지 않고, 대상자에만 표시 |
{'author': '학사종합지원센터', 'date': '2025.04.04', 'doc_type': '학사공지', 'ntt_id': '223498', 'title': '[공통] 후기이중전공 신청 공고', 'url': 'https://www.hufs.ac.kr/bbs/hufs/2181/223498/artclView.do'}
[공통] 후기이중전공 신청 공고

[공통] 후기이중